In [6]:
import pandas as pd
import seaborn as sns
import re
import language_tool_python
import ast
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import ssl
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
tweets = pd.read_csv("tweets_data.csv")
tweets['account.type'] = 'human'
tweets.loc[tweets['isBot'] == 1, 'account.type'] = 'bot'
tweets.drop('isBot', axis = 1, inplace = True)
tweets

,Unnamed: 0,text,account.type
0,0,YEA now that note GOOD,bot
1,1,Listen to This Charming Man by The Smiths htt...,human
2,2,wish i can i would be seeing other hoes on the...,bot
3,3,The decade in the significantly easier schedul...,bot
4,4,"""Theim class=\""alignnone size-full wp-image-60...",bot
...,...,...,...
25567,2553,“The best kept secret,bot
25568,2554,Love the Choose your own adventure style of th...,human
25569,2555,JOIN OUR TEAM: Sneezing,bot
25570,2556,These deeply discounted 256GB SanDisk flash dr...,human


In [8]:

tweets['hashtags'] =''
tweets['mentions'] =''
for word in range(len(tweets)):
    tweet = tweets["text"][word]
    tweet.replace('\n', " ")
    word_list = tweet.split(' ')
    regex = "#(\w+)"
    regex2 = "@(\w+)"
    hashtag_list = re.findall(regex, tweet)
    mentions_list = re.findall(regex2, tweet)
    tweets["hashtags"][word] = hashtag_list
    tweets["mentions"][word] = mentions_list

C:\Users\veena\AppData\Local\Temp\ipykernel_35024\2451328056.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["hashtags"][word] = hashtag_list
C:\Users\veena\AppData\Local\Temp\ipykernel_35024\2451328056.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["mentions"][word] = mentions_list


In [9]:
tweets

,Unnamed: 0,text,account.type,hashtags,mentions
0,0,YEA now that note GOOD,bot,[],[]
1,1,Listen to This Charming Man by The Smiths htt...,human,[],[]
2,2,wish i can i would be seeing other hoes on the...,bot,[],[]
3,3,The decade in the significantly easier schedul...,bot,[],[]
4,4,"""Theim class=\""alignnone size-full wp-image-60...",bot,[],[]
...,...,...,...,...,...
25567,2553,“The best kept secret,bot,[],[]
25568,2554,Love the Choose your own adventure style of th...,human,[],[]
25569,2555,JOIN OUR TEAM: Sneezing,bot,[],[]
25570,2556,These deeply discounted 256GB SanDisk flash dr...,human,[],[pcworld]


In [10]:

tool = language_tool_python.LanguageTool('en-US')
tweets['no_of_grammar_errors'] = ''
for word in range(len(tweets)):
    cap = tweets["text"][word]
    word_list = cap.split(' ')
    tweets['no_of_grammar_errors'][word] = len(tool.check(cap))
tweets

ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.

In [ ]:
tweets

In [ ]:
tweets['num_emojis'] = ''
tweets['emojis'] = ''
for word in range(len(tweets)):
    tweet = tweets["text"][word]
    tweet.replace('\n', " ")
    regex = "<U+.*?>"
    emoji_list = re.findall(regex, tweet)
    tweets["num_emojis"][word] = len(emoji_list)
    tweets["emojis"][word] = emoji_list

In [ ]:

tweets['mentions_count'] = tweets['mentions'].apply(lambda x: len(x))
tweets['hashtags_count'] = tweets['hashtags'].apply(lambda x: len(x))
human_tweets = tweets[tweets['account.type']== 'human']
bot_tweets = tweets[tweets['account.type']== 'bot']
tweets

Grammar Errors

In [ ]:
sns.barplot(data=tweets, x="account.type", y="no_of_grammar_errors")

In [ ]:
sns.stripplot(data=tweets, x="account.type", y="no_of_grammar_errors")

Mentions

In [ ]:
sns.barplot(data=tweets, x="account.type", y="mentions_count")

In [ ]:
sns.stripplot(data=tweets, x="account.type", y="mentions_count")

Hashtags

In [ ]:
sns.barplot(data=tweets, x="account.type", y="hashtags_count")

In [ ]:
sns.stripplot(data=tweets, x="account.type", y="hashtags_count")

Emojis

In [ ]:
sns.barplot(data=tweets, x="account.type", y="num_emojis")

In [ ]:
sns.stripplot(data=tweets, x="account.type", y="num_emojis")

In [ ]:
human_tweets['no_of_grammar_errors'] = human_tweets['no_of_grammar_errors'].apply(int)
human_tweets['num_emojis'] = human_tweets['num_emojis'].apply(int)
relevant_values_human = human_tweets[['no_of_grammar_errors','hashtags_count','num_emojis','mentions_count']]
sns.heatmap(relevant_values_human.corr(),cmap ='coolwarm', annot=True,)

In [ ]:
bot_tweets['no_of_grammar_errors'] = bot_tweets['no_of_grammar_errors'].apply(int)
bot_tweets['num_emojis'] = bot_tweets['num_emojis'].apply(int)
relevant_values_bot = bot_tweets[['no_of_grammar_errors','hashtags_count','num_emojis','mentions_count']]
sns.heatmap(relevant_values_bot.corr(),cmap ='coolwarm', annot=True,)

In [ ]:
sns.pairplot(relevant_values_human) 

In [ ]:
sns.pairplot(relevant_values_bot)

In [ ]:
# import textblob
# from textblob import Word

# tweets['incorrect_spell_count'] = ''
# tweets['incorrect_spell_words'] = ''
# for word in range(len(tweets)):
#     cap = tweets["text"][word]
#     word_list = cap.split(' ')
#     incorrect_words = []
#     for i in word_list:
#         if Word(i).spellcheck()[0][1] < 1.0:
#             incorrect_words.append(i)
#     # new_caption = " ".join(correct_words)
#     # captions["clean_captions"][word] = new_caption
#     tweets['incorrect_spell_words'][word] = incorrect_words
#     tweets['incorrect_spell_count'][word] = len(incorrect_words)

In [ ]:
tweets

In [ ]:
def remove_numbers(df):
    for word in range(len(df)):
        pattern = '[0-9]'
        tweet_clean = df['text'][word].split(' ')
        tweet_clean = [re.sub(pattern, '', i) for i in tweet_clean]
        new_tweet = " ".join(tweet_clean)
        df['text'][word] = new_tweet
    return df
tweets = remove_numbers(tweets)

In [ ]:
bot_tweets = tweets.loc[tweets['account.type'] == 'bot']
human_tweets = tweets.loc[tweets['account.type'] == 'human']
bot_tweets_text = bot_tweets["text"]
human_tweets_text = human_tweets["text"]
human_tweets_text

In [ ]:

ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('all')

bot_tweets_tokens = []

lemma = WordNetLemmatizer()
for sentence in bot_tweets_text:
    bot_tweets_tokens.append(word_tokenize(sentence))
bot_tweets_tokens_2 = []
for sentence in bot_tweets_tokens:
    sentence2 = []
    for word in sentence:
        if word not in stopwords.words('english'):
            sentence2.append(word)
    bot_tweets_tokens_2.append(sentence2)
bot_tweets_tokens_2
bot_tweets_tokens_3 = []
for sentence in bot_tweets_tokens_2:
    sentence3 = []
    for word in sentence:
        sentence3.append(lemma.lemmatize(word))
    bot_tweets_tokens_3.append(sentence3)
len(bot_tweets_tokens_3)
bot_tweets_tokens_4 = []
for sentence in bot_tweets_tokens_3:
    sentence4 = []
    for word in sentence:
        sentence4.append(PorterStemmer().stem(word))
    bot_tweets_tokens_4.append(sentence4)

In [ ]:
human_tweets_tokens = []
for sentence in human_tweets_text:
    human_tweets_tokens.append(word_tokenize(sentence))
human_tweets_tokens_2 = []
for sentence in human_tweets_tokens:
    sentence2 = []
    for word in sentence:
        if word not in stopwords.words('english'):
            sentence2.append(word)
    human_tweets_tokens_2.append(sentence2)
human_tweets_tokens_2
human_tweets_tokens_3 = []
for sentence in human_tweets_tokens_2:
    sentence3 = []
    for word in sentence:
        sentence3.append(lemma.lemmatize(word))
    human_tweets_tokens_3.append(sentence3)
len(human_tweets_tokens_3)
human_tweets_tokens_4 = []
for sentence in human_tweets_tokens_3:
    sentence4 = []
    for word in sentence:
        sentence4.append(PorterStemmer().stem(word))
    human_tweets_tokens_4.append(sentence4)

In [ ]:

#print(bot_tweets_tokens_4)
human_tweets_str = []
for i in range(len(human_tweets_tokens_4)):
    human_tweets_str.append(" ".join(human_tweets_tokens_4[i]))
vectorizer = CountVectorizer().fit(human_tweets_str)
human_tweets_tfidf = TfidfTransformer().fit_transform(vectorizer.transform(human_tweets_str))

bot_tweets_str = []
for i in range(len(bot_tweets_tokens_4)):
    bot_tweets_str.append(" ".join(bot_tweets_tokens_4[i]))
bot_tweets_tfidf = TfidfTransformer().fit_transform(vectorizer.transform(bot_tweets_str))
(human_tweets_tfidf, bot_tweets_tfidf)

In [ ]:
# Compare Vocabulary size

human_tweets_str = []
for i in range(len(human_tweets_tokens_4)):
    human_tweets_str.append(" ".join(human_tweets_tokens_4[i]))
human_tweets_countvec = CountVectorizer().fit_transform(human_tweets_str)

bot_tweets_str = []
for i in range(len(bot_tweets_tokens_4)):
    bot_tweets_str.append(" ".join(bot_tweets_tokens_4[i]))
bot_tweets_countvec = CountVectorizer().fit_transform(bot_tweets_str)

(human_tweets_countvec, bot_tweets_countvec)

In [ ]:

new_matrix = human_tweets_tfidf * csr_matrix.transpose(bot_tweets_tfidf)
new_matrix



fig, ax1 = plt.subplots(1,1)
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
im = ax1.imshow(new_matrix.todense(), cmap="copper_r", vmin=0, vmax = 0.05)
plt.colorbar(im)
plt.show()

In [ ]:
# Dot Product between Bot and Bot Tweets
dot_prod_matrix_bots = bot_tweets_tfidf * csr_matrix.transpose(bot_tweets_tfidf)
# sns.heatmap(dot_prod_matrix_bots.todense(), vmin=0, vmax=0.2)
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.title('Dot-Product between bot and bot tweets', fontdict={'fontsize': 10,
 'fontweight' : plt.rcParams['axes.titleweight'],
 'verticalalignment': 'baseline'})
plt.xlabel('Bot')
plt.ylabel('Bot') 
im = plt.imshow(dot_prod_matrix_bots.todense(), cmap="copper_r", vmin=0, vmax = 0.05)
plt.colorbar(im)
plt.show()

In [ ]:
# Dot Product between Human and Human Tweets
dot_prod_matrix_human = human_tweets_tfidf * csr_matrix.transpose(human_tweets_tfidf)
# sns.heatmap(dot_prod_matrix_human.todense(), vmin=0, vmax=0.2)
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
im = plt.imshow(dot_prod_matrix_human.todense(), cmap="copper_r", vmin=0, vmax = 0.05)
plt.colorbar(im)
plt.show()

In [ ]:
dot_prod_matrix_human

In [31]:
print("% of zero values in human-human dot product matrix:", (11636*11634 - 13411199) /(11636*11634))
print("% of zero values in bot-bot dot product matrix:", (11634*11634 - 16463632) /(11634*11634))
print("% of zero values in human-bot dot product matrix:", (11636*11636 - 31930203) /(11636*11636))

% of zero values in human-human dot product matrix: 0.9009316716871573
% of zero values in bot-bot dot product matrix: 0.878362487240114
% of zero values in human-bot dot product matrix: 0.7641726045849813
